<a href="https://colab.research.google.com/github/radhika004/CustomerSupportTickets/blob/main/Recurring_Issue_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import csv
import random
import requests

# Here I tried to automate responses for issues:
# 1. Returns and Exchanges
# 2. Technical Support
# 3. Billing Issues

LLAMA_API_KEY = "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"
# templates
response_templates = {
    'Returns and Exchanges': """
    Subject: Your Return/Exchange Request

    Dear {customer_name},

    Thank you for reaching out to us. We have received your request for a return/exchange of the product: {product_name}.

    We understand the importance of this matter and would like to inform you that your request is under review. Please ensure that the product is unused and in its original packaging. If it meets our criteria, we will process the return/exchange within the next {processing_time} business days.

    If you have any questions or require further assistance, feel free to reach out.

    Thank you for your patience!

    Best regards,
    {company_name} Support Team
    """,
    'Technical Support': """
    Subject: Your Technical Support Request

    Dear {customer_name},

    Thank you for reaching out to our Technical Support team. We understand you're experiencing an issue with {issue_description}.

    Our team is actively working on resolving this. We will update you with the progress shortly. In the meantime, please follow these steps to assist in troubleshooting:

    1. {try to reinstall the software}
    2. {Go through the documentation}
    3. {Check if there any error fix steps on our official website }

    If these steps do not resolve the issue, please let us know, and we will proceed with further troubleshooting.

    Best regards,
    {company_name} Technical Support Team
    """,
    'Billing Issues': """
    Subject: Billing Issue

    Dear {customer_name},

    Thank you for contacting us about the billing issue. We are sorry for the inconvenience caused.

    We have reviewed your account, and it seems the issue is related to {billing_issue}. Please allow us {processing_time} business days to investigate further. We will notify you once the issue has been resolved.

    If you have any urgent concerns, feel free to contact us.

    Best regards,
    {company_name} Billing Support
    """
}

# Categories to pick from
types = ['Incident', 'Request', 'Problem', 'Change']
queues = ['Technical Support', 'Product Support', 'Sales and Pre-Sales', 'Returns and Exchanges', 'IT Support', 'Billing and Payments', 'Customer Service', 'Service Outages and Maintenance', 'Human Resources', 'General Inquiry']
priorities = ['high', 'low', 'medium']
tags = ['Service Outage', 'Technical Support', 'Data Breach', 'Customer Service', 'Returns and Exchanges', 'Login Issue', 'Software Bug', 'Billing Issue', 'IT Support', 'Urgent Issue', 'Product Support', 'Service Disruption', 'Network Issue', 'Hardware Failure', 'General Inquiry', 'System Maintenance', 'Sales Inquiry', 'Password Reset', 'Feature Request', 'Performance Tuning', 'Critical Failure', 'Service Maintenance', 'Consultoría Informática', 'Consultation Request']
business_types = ['IT Services', 'Tech Online Store', 'IT Consulting Firm', 'Software Development Company', 'IT Consulting Service']

# Function to generate AI-based response using LLaMA 3 API
def generate_ai_response_llama(issue_description):
    payload = {
        "model": "llama3-8b-8192",
        "inputs": f"""
        You are a customer care assistant. Generate a professional response based on the following issue description:
        "{issue_description}"
        Include a helpful subject and body for the email.
        """,
        "parameters": {
            "max_tokens": 300,
            "temperature": 0.7,
        }
    }
    headers = {
        "Authorization": f"Bearer {LLAMA_API_KEY}",
        "Content-Type": "application/json",
    }

    try:
        response = requests.post(LLAMA_API_URL, json=payload, headers=headers)
        response.raise_for_status()
        return response.json().get("choices", [{}])[0].get("text", "").strip()
    except Exception as e:
        return f"Error generating AI response: {e}"

# Function to predict fields dynamically using LLaMA 3
def predict_fields_ai_llama(issue_description):
    payload = {
        "model": "llama3-8b-8192",
        "inputs": f"""
        Based on the issue description: "{issue_description}", predict the following fields:
        - Type (Incident, Request, Problem, Change)
        - Queue (Technical Support, Product Support, Sales, etc.)
        - Priority (High, Medium, Low)
        - Tags (Tag 1 to Tag 5 from the list: Service Outage, Technical Support, Login Issue, etc.)
        - Business Type (IT Services, Tech Online Store, etc.)
        Provide the predictions as a JSON object.
        """,
        "parameters": {
            "max_tokens": 200,
            "temperature": 0.5,
        }
    }
    headers = {
        "Authorization": f"Bearer {LLAMA_API_KEY}",
        "Content-Type": "application/json",
    }

    try:
        response = requests.post(LLAMA_API_URL, json=payload, headers=headers)
        response.raise_for_status()
        return eval(response.json().get("choices", [{}])[0].get("text", "").strip())
    except Exception as e:
        return {
            "type": random.choice(types),
            "queue": random.choice(queues),
            "priority": random.choice(priorities),
            "tags": random.sample(tags, 3),
            "business_type": random.choice(business_types)
        }

# Function to append data to CSV file
def append_to_csv(file_name, data):
    with open(file_name, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(data)

# Function to process the issue based on user input
def process_issue(file_name, issue_description, issue_details, user_category=None):
    ai_response = generate_ai_response_llama(issue_description)
    predicted_fields = predict_fields_ai_llama(issue_description)

    # Choose category-specific template or generate random responses
    if user_category and user_category in response_templates:
        template = response_templates[user_category]
        if user_category == 'Returns and Exchanges':
            response = template.format(
                customer_name=issue_details.get("customer_name", "Customer"),
                product_name=issue_details.get("product_name", "Product"),
                processing_time="7",  # Can be dynamic
                company_name="Your Company"
            )
        elif user_category == 'Technical Support':
            response = template.format(
                customer_name=issue_details.get("customer_name", "Customer"),
                issue_description=issue_description,
                step_1="Restart your device",
                step_2="Check your internet connection",
                step_3="Update your software",
                company_name="Your Company"
            )
        elif user_category == 'Billing Issues':
            response = template.format(
                customer_name=issue_details.get("customer_name", "Customer"),
                billing_issue="incorrect charge",
                processing_time="5",  # Can be dynamic
                company_name="Your Company"
            )
    else:
        # Default generic response if no user category provided
        response = "Thank you for contacting us. We will address your issue shortly."

    # Prepare row data for CSV
    row_data = [
        issue_details.get("product_name", ""),
        issue_details.get("product_id", ""),
        issue_details.get("bill_no", ""),
        issue_description,
        ai_response,  # AI-generated response
        response,  # Predefined response or dynamic response
        predicted_fields["type"],
        predicted_fields["queue"],
        predicted_fields["priority"],
        predicted_fields["business_type"]
    ] + predicted_fields["tags"]

    # Append data to CSV
    append_to_csv(file_name, row_data)

    # Display filled columns
    print("Updated Fields in the CSV:")
    print(f"Product Name: {issue_details.get('product_name', '')}")
    print(f"Product ID: {issue_details.get('product_id', '')}")
    print(f"Bill No: {issue_details.get('bill_no', '')}")
    print(f"Issue Description: {issue_description}")
    print(f"AI Response: {ai_response}")
    print(f"Response: {response}")
    print(f"Type: {predicted_fields['type']}")
    print(f"Queue: {predicted_fields['queue']}")
    print(f"Priority: {predicted_fields['priority']}")
    print(f"Business Type: {predicted_fields['business_type']}")
    print(f"Tags: {', '.join(predicted_fields['tags'])}")
    print("Issue processed and added to CSV successfully.")

# Example usage
if __name__ == "__main__":
    issue_description = "The product I received has a cracked screen, and I would like to request a replacement."
    issue_details = {
        "product_name": "Mac book ",
        "product_id": "1294905804986",
        "bill_no": "67890",
        "customer_name": "Avinash Kumar"
    }
    user_category = "Returns and Exchanges"
    process_issue("/customer_support.csv", issue_description, issue_details, user_category)


Updated Fields in the CSV:
Product Name: Mac book 
Product ID: 1294905804986
Bill No: 67890
Issue Description: The product I received has a cracked screen, and I would like to request a replacement.
AI Response: Error generating AI response: name 'LLAMA_API_URL' is not defined
Response: 
    Subject: Your Return/Exchange Request

    Dear Avinash Kumar,

    Thank you for reaching out to us. We have received your request for a return/exchange of the product: Mac book . 

    We understand the importance of this matter and would like to inform you that your request is under review. Please ensure that the product is unused and in its original packaging. If it meets our criteria, we will process the return/exchange within the next 7 business days.

    If you have any questions or require further assistance, feel free to reach out.

    Thank you for your patience!

    Best regards,
    Your Company Support Team
    
Type: Change
Queue: Sales and Pre-Sales
Priority: high
Business Type: S

still its not giving totally correct output because there is wrong prediction about type,queue,priority..etc